In [ ]:
#packages we need
using DifferentialEquations
using Plots 
using CSV
using Distributions
using Random
using DataFrames
using JLD2
using DelimitedFiles
using OrdinaryDiffEq
using LinearAlgebra
using FiniteDiff
using FileIO

In [ ]:
Random.seed!(1234) #for reproducibility

In [ ]:
#set up where CSV2Julia is
locationOfCSV2Julia="csv2model-multiscale.py"

#identify the three CSV sheets that describe the model
rateLawsFile="rateLaws_plus_myc3_NS_Mass2.csv"
reactionsFile="reactions_plus_myc3_NS_Mass2.csv"
#parametersFile="parameters_plus_myc.csv" # WT
#parametersFile="parameters_plus_myc_nodistGM.csv" # no distribution of GM
#parametersFile="parameters_plus_myc_halfmyc.csv" # WT, k1_myc = 0.462
#parametersFile="parameters_plus_myc_mut.csv" # myc x1.5
#parametersFile="parameters_plus_myc_mut_CycA_exp.csv" # myc x1.5 CycA x1.5
#parametersFile="parameters_plus_myc_x5_CycA_exp.csv" # myc x5 CycA x1.5
parametersFile="parameters_plus_myc_x20_CycA_exp.csv" # myc x20 CycA x1.5
#parametersFile="parameters_plus_myc_mut_p27_mut.csv" # myc x1.5 p27 x1.5
#parametersFile="parameters_plus_myc_mut_five.csv" # myc x5
#parametersFile="parameters_plus_myc_E2F_Rb_diss.csv" # E2F_Rb_diss x1.5
#parametersFile="parameters_plus_myc_E2F_bind_Rb.csv" # E2F_bind_Rb x1.5
#parametersFile="parameters_plus_myc_Cdc20_exp.csv" # Cdc20_exp x 1.5
#parametersFile="parameters_plus_myc_CycA_exp.csv" # CycA_exp x 1.5
#parametersFile="parameters_plus_myc_CycD_exp.csv" # CycD_exp x 1.5
#parametersFile="parameters_plus_myc_CycE_exp.csv" # CycE_exp x 1.5
#parametersFile="parameters_plus_myc_downreg.csv" # myc del

#CV =  coefficient of variation
preCV=0.11 # as reported here: https://www.pnas.org/content/115/12/E2888
#numberOfCells=50

maxTimeSS=100000.0
maxTimeTC=24*60.0

DISincrease=0.000001

## set time to solve
TCLength=300*60
## set max attempts to find steady state
maximumAttemptsAtSS=10

In [ ]:
first_cell = 1
last_cell = 1000

In [ ]:
#mut_folder="WT"
#mut_folder="WT_fix2"
#mut_folder="WT_test"
#mut_folder="WT_fix"

#mut_folder="myc_mut"
#mut_folder="myc_mut_test"
#mut_folder="myc_del"
#mut_folder="myc_p27_mut"
#mut_folder="myc_mut_CycA_exp"
#mut_folder="myc_x5_CycA_exp"
mut_folder="myc_x20_CycA_exp"
#mut_folder="myc_mut_rpt"
#mut_folder="E2F_Rb_diss"
#mut_folder="E2F_bind_Rb"
#mut_folder="Cdc20_exp"
#mut_folder="CycA_exp"
#mut_folder="CycD_exp"
#mut_folder="CycE_exp"
#mut_folder="myc_mut_five"

In [ ]:
mkpath("odeFiles/"*mut_folder*"/")
mkpath("paramFiles/"*mut_folder*"/")

In [ ]:
include("fixSpecies.jl")

In [ ]:
#distributes a bunch of ODE functions and imports them with names that are specific to each cell
for i in first_cell:last_cell
    thisParamDF = DataFrame(CSV.File(parametersFile,types=Dict(:parameter=>String, :value=>String, :distribute=>Int64)))
    println("starting cell: "*string(i));flush(stdout);
    thisDist=TruncatedNormal(1.0, preCV,0,Inf)
    #now lets loop through every parameter and distribute it if there is a 1 in the DF in the distribute column
    for j in 1:size(thisParamDF,1)
        if thisParamDF[j,3]==1
            x = rand(thisDist, 1)
            oldParam=parse(Float64,thisParamDF[j,2])
            thisParamDF[j,2]=string(oldParam.*x[1])
        end
    end
    #now write this cell's CSV file to a folder of cells
    CSV.write("parameters_cell_"*string(i)*".csv", thisParamDF)
    println("generated CSV file for cell: "*string(i));flush(stdout);
    modelFile="odeModel_"*string(i)*".jl"
    arguments=[reactionsFile, "parameters_cell_"*string(i)*".csv", rateLawsFile, modelFile, "inline"]
    cmd=`python3 $locationOfCSV2Julia $arguments`
    run(cmd)
    #fixSpecies("odeModel_"*string(i)*".jl","odeModel_"*string(i)*".jl", 23)
    mv("odeModel_"*string(i)*".jl", "odeFiles/"*mut_folder*"/odeModel_"*string(i)*".jl", force=true)
    mv("parameters_cell_"*string(i)*".csv", "paramFiles/"*mut_folder*"/parameters_cell_"*string(i)*".csv", force=true)
    modelFile="odeFiles/"*mut_folder*"/odeModel_"*string(i)*".jl"
    include(modelFile)
end

In [ ]:
locationOfVariableNames="variableNames.jl"
include(locationOfVariableNames)
indexOfMass=findfirst(x->"Mass"==x,syms)
indexOfGM=findfirst(x->"GM"==x,syms)
indexNewSwitch=findfirst(x->"newSwitch"==x,syms)
indexOfRb=findfirst(x->"Rb"==x,syms)
indexOfE2FRb=findfirst(x->"E2F_Rb"==x,syms)
indexOfpE2FRb=findfirst(x->"pE2F_Rb"==x,syms)
indexOfHypoPRb=findfirst(x->"HypoP_Rb"==x,syms)
indexOfSwitch=findfirst(x->"r31switch"==x,syms)
indexOfCdh1=findfirst(x->"Cdh1"==x,syms)
indexOfMassTracker1=findfirst(x->"previousMass1"==x,syms)
indexOfMassTracker2=findfirst(x->"previousMass2"==x,syms)
indexOfMassTracker3=findfirst(x->"previousMass3"==x,syms)
indexOfMassTracker4=findfirst(x->"previousMass4"==x,syms)
indexOfMassTracker5=findfirst(x->"previousMass5"==x,syms)
indexOfMassTrackerTime1=findfirst(x->"previousMassTime1"==x,syms)
indexOfMassTrackerTime2=findfirst(x->"previousMassTime2"==x,syms)
indexOfMassTrackerTime3=findfirst(x->"previousMassTime3"==x,syms)
indexOfMassTrackerTime4=findfirst(x->"previousMassTime4"==x,syms)
indexOfMassTrackerTime5=findfirst(x->"previousMassTime5"==x,syms)

In [ ]:
function condition(cellcycle,t,integrator) # Event when event_f(u,t) == 0
      #numberator: Rb + E2F:Rb +  pE2F:Rb
      #denominator: Rb + E2F:Rb +  pE2F:Rb + hypophosphorylated Rb
      #numberator: Rb + E2F:Rb +  pE2F:Rb
      #denominator: Rb + E2F:Rb +  pE2F:Rb + hypophosphorylated Rb
      numerator= cellcycle[indexOfRb]+cellcycle[indexOfE2FRb]+cellcycle[indexOfpE2FRb];  #ACTIVE RB
      denominator = numerator+cellcycle[indexOfHypoPRb]; #TOTAL RB

      #GM growth (biosynthesis of ribosomes and all necessary machinery)
      #previous iterations had numerator/denominator<0.8
      #diffEq.jl requires a function that hits 0
      (numerator/denominator)-0.8
end

function affect!(integrator)
  print("r31 event to 0\n")
  #update r31Switch to 0
  integrator.u[indexOfSwitch] = 0
end

function affectNeg!(integrator)
  print("r31 event to 1\n")
  #update r31Switch to 1
  integrator.u[indexOfSwitch] = 1
end

function conditionCdh(cellcycle,t,integrator) # Event when event_f(u,t) == 0

      #triggers when Cdh1 crosses 0.2 in the positive direction
      cellcycle[indexOfCdh1]-0.2
end

In [ ]:
## all equal function
function allequal(obj)
    local x
    isfirst = true
    for i in obj
        if isfirst
            x = i
            isfirst = false
        else
            isequal(x, i) || return false
        end
    end
    return true
end

In [ ]:
function affectCdh!(integrator)

  #triggers when Cdh1 crosses 0.2 in the positive direction
  #update Mass to 0.5* mass

#   # get mass value just before divide
#   mass_cyc = integrator.u[indexOfMass]
#   time = integrator.t
#   # push mass value to array
#   push!(mass_array, mass_cyc)
#   # push mass peak time to array
#   push!(mass_peak_time, time)
#   # if mass_array > 5, round mass values, and compare last 5
#   n=size(mass_array,1)
#   if n>6
#     mass_array_last = last(mass_array, 6)
#     mass_array_rnd = round.(mass_array_last, digits = 3)
#     if allequal(mass_array_rnd) == true
#       terminate!(integrator)
#       return(mass_array, mass_peak_time)
#     else
   integrator.u[indexOfMass] = 0.5*integrator.u[indexOfMass]
#   #Gm to 0.5 * Gm
   integrator.u[indexOfGM]=0.5*integrator.u[indexOfGM]
# end
# end
  if integrator.u[indexOfMass]<0.5
    #print("Mass dropped below 0.5")
    integrator.u[indexNewSwitch]=0
  else
    #print("new switch 1 during division\n")
    integrator.u[indexNewSwitch]=1
  end
 
    #store the latest peak mass in the last spot and shuffle every entry down one.
    integrator.u[indexOfMassTracker5]=integrator.u[indexOfMassTracker4]
    integrator.u[indexOfMassTracker4]=integrator.u[indexOfMassTracker3]
    integrator.u[indexOfMassTracker3]=integrator.u[indexOfMassTracker2]
    integrator.u[indexOfMassTracker2]=integrator.u[indexOfMassTracker1]
    integrator.u[indexOfMassTracker1]=round(integrator.u[indexOfMass], digits=3)
    integrator.u[indexOfMassTrackerTime5]=integrator.u[indexOfMassTrackerTime4]
    integrator.u[indexOfMassTrackerTime4]=integrator.u[indexOfMassTrackerTime3]
    integrator.u[indexOfMassTrackerTime3]=integrator.u[indexOfMassTrackerTime2]
    integrator.u[indexOfMassTrackerTime2]=integrator.u[indexOfMassTrackerTime1]
    integrator.u[indexOfMassTrackerTime1]=integrator.t
    lastPeaks=[integrator.u[indexOfMassTracker5] integrator.u[indexOfMassTracker4] integrator.u[indexOfMassTracker3] integrator.u[indexOfMassTracker2] integrator.u[indexOfMassTracker1]]
    if allequal(lastPeaks)
        
       terminate!(integrator)
    end

 #print("division event\n")
end

In [ ]:
# extra functions to stop the cell cycling when mass drops below 0.5

function conditionMass(cellcycle,t,integrator) # Event when event_f(u,t) == 0
      #triggers when mass crosses 0.5
      cellcycle[indexOfMass]-0.5
end

function affectMass!(integrator)
    integrator.u[indexNewSwitch] = 1
    #print("new switch 1\n")
end

function affectMassNeg!(integrator)
    integrator.u[indexNewSwitch] = 0
    #print("new switch 0\n")
end

In [ ]:
#use the callbacks described above to trigger events
cb = ContinuousCallback(condition,affect!,affectNeg!);
cb2 = ContinuousCallback(conditionCdh,affectCdh!,nothing);
cb3 = ContinuousCallback(conditionMass,affectMass!,affectMassNeg!);
cbs=CallbackSet(cb,cb2,cb3);

In [ ]:
global syms=Symbol.(syms);

In [ ]:
function initConditionsCellCycle(y0,syms)
   #units: M 
   y0[findfirst(isequal("ERG"),syms)]=0.0121809 
   y0[findfirst(isequal("p27_cycA_Cdk2"),syms)]=0.0356927
   y0[findfirst(isequal("p27"),syms)]=0.00922806
   y0[findfirst(isequal("Cdc20"),syms)]=0.00220177 
   y0[findfirst(isequal("p27_cycE_Cdk2"),syms)]=0.000542587
   y0[findfirst(isequal("cycE"),syms)]=0.0229112 
   y0[findfirst(isequal("cycA"),syms)]=1.4094 
   y0[findfirst(isequal("cycB"),syms)]=2.72898
   y0[findfirst(isequal("p27_cycD_Cdk2"),syms)]=0.010976 
   y0[findfirst(isequal("cycD"),syms)]=0.43929 
   y0[findfirst(isequal("Cdh1"),syms)]=0.000653278 
   y0[findfirst(isequal("DRG"),syms)]=0.900533
   y0[findfirst(isequal("PPX"),syms)]=1.0
   y0[findfirst(isequal("IEP"),syms)]=0.154655
   y0[findfirst(isequal("Cdc20t"),syms)]=2.36733
   y0[findfirst(isequal("E2F_Rb"),syms)]=0.00478911
   y0[findfirst(isequal("E2F"),syms)]=0.989986
   y0[findfirst(isequal("HypoP_Rb"),syms)]=9.97574
   y0[findfirst(isequal("pE2F_Rb"),syms)]=0.0192822
   y0[findfirst(isequal("pE2F"),syms)]=3.98594
   y0[findfirst(isequal("GM"),syms)]=1.35565
   y0[findfirst(isequal("r31switch"),syms)]=1
   y0[findfirst(isequal("cMyc"),syms)]=80  ## orig 40
   y0[findfirst(isequal("Mass"),syms)]=1
   y0[findfirst(isequal("newSwitch"),syms)]=1
   y0[findfirst(isequal("Rb"),syms)]=0
   y0[findfirst(isequal("cMyct"),syms)]=40
   return y0
end

In [ ]:
#now we'll actually solve the models

import Base.Threads

In [ ]:
mkpath("outputFiles_cell_cycle/"*mut_folder*"/")
mkpath("outputFiles_cell_cycle/"*mut_folder*"/LastCellCyclecsv/")
mkpath("outputFiles_cell_cycle/"*mut_folder*"/quiescent/")
mkpath("outputFiles_cell_cycle/"*mut_folder*"/plot_times/")

In [ ]:
### NEW FUNCTION

function Get_CC_transitions(sol_SS, time_stamp_SS)
  Cdh1_col = columnindex(sol_SS, :Cdh1)
  cycA_col = columnindex(sol_SS, :cycA)
  cycB_col = columnindex(sol_SS, :cycB)
  cycE_col = columnindex(sol_SS, :cycE)
  Mass_col = columnindex(sol_SS, :Mass)

  sol_SS_cc = sol_SS[:, [Cdh1_col,cycA_col,cycB_col,cycE_col,Mass_col]]

  # G1 -> S boundary when cycB is > 0 (or close to 0)cycB_peak_df = convert_colname(max_cycB[2])
  # S to G2 when CycE is gone but before peak in CycA
  # G2 to M at peak of CycB before rapid decrease in CycB, CycA should already be decreasing and about half way gone.

  SS_zero = convert(Int64, round(time_stamp_SS[1], digits=0))

  ##G1/S
  max_Cdh1 = findmax(sol_SS_cc[:,1])
  submax_Cdh1 = (max_Cdh1[1]/100)*95
  submax_Cdh1 = findfirst(x->x>submax_Cdh1, sol_SS_cc[:,1])
  G1S_df = sol_SS_cc[submax_Cdh1:nrow(sol_SS_cc),:]
  col_Cdh1 = G1S_df[:,1]  
  half_max_Cdh1 = max_Cdh1[1]*0.5
  G1S=findfirst(x->x<half_max_Cdh1,col_Cdh1)
  G1S_t = convert(Int64, round(time_stamp_SS[G1S+submax_Cdh1],digits=0))
  G1S_t = G1S_t - SS_zero
    
  ## S/G2
  G1St = SS_zero+G1S_t
  G1St_TS = findfirst(x->x>G1St, time_stamp_SS)
  # get max value of cycA
  colcycA = sol_SS_cc[:,2]
  max_cycA = findmax(colcycA[G1St_TS:end])
  # get time when cycA is max - 2.5%
  submaxA = (max_cycA[1]/100)*97.5
  submaxA = findfirst(x->x>submaxA,colcycA)
  SG2_t = convert(Int64, round(time_stamp_SS[submaxA],digits=0))
  SG2_t = SG2_t - SS_zero

  ## G2/M
  SG2t = SS_zero+SG2_t
  SG2t_TS = findfirst(x->x>SG2t, time_stamp_SS)
  col_cycB = sol_SS_cc[:,3]
  max_cycB = findmax(col_cycB[SG2t_TS:end])
  #submaxB = (max_cycB[1]/100)*97.5
  max_cycB = findfirst(x->x==max_cycB[1],col_cycB)
  G2M_time = convert(Int64, round(time_stamp_SS[max_cycB],digits=0))
  G2M_t = G2M_time - SS_zero

  ## M
  M_t = convert(Int64, round(time_stamp_SS[end],digits=0)) - SS_zero

  return(G1S_t,SG2_t,G2M_t,M_t)

end

In [ ]:
global colnames=["G1S", "SG2", "G2M", "M"]

In [ ]:
### Save quiescent cells, calculate steady state

function solveCell(i)
    println("Starting solving cell: "*string(i)*".")
    #figure out the name of this cell's ode file
    odeName="odeModel_"*string(i)
    myFun=getfield(Main,Symbol(odeName))

    f=ODEFunction(myFun,syms=Symbol.(syms))
    y0=zeros(length(syms))
    y0=initConditionsCellCycle(y0,String.(syms))
    prob=ODEProblem(f,y0,(0.0,maxTimeSS))
    println("about to solve cell: "*string(i))
    
    sol = solve(prob,callback=cbs, abstol=1e-5,reltol=1e-3,saveat=1.0)
    sol_df = DataFrame(sol', syms)
    sol_NS = sol_df[!, :newSwitch]
    
    if      isequal(sol_NS[end], 1.0) == false
            println("cell is in a quiescent state")
            CSV.write("outputFiles_cell_cycle/"*mut_folder*"/quiescent/sol_df"*string(i)*".csv", sol_df)
        
    elseif  sol.t[end]==maxTimeSS   
            println("Cell: "*string(i)*" ended without finding a steady state in time.")
    
    else    println("Cell: "*string(i)*" reached a steady state.")

            ### get steady state
            sol_df = DataFrame(sol', syms)
            time_stamp = sol.t
            sol_df[!, :time_stamp] = time_stamp
            indexOfLastCycleStart=findfirst(x->x>sol[indexOfMassTrackerTime2,end],sol.t)
            sol_SS = sol_df[indexOfLastCycleStart:end, :]

            # get SS time stamp
            time_stamp_SS = sol_SS[:, :time_stamp]

            ### Calculate times for cell cycle transitions
            plot_times = DataFrame(Array(collect(Get_CC_transitions(sol_SS, time_stamp_SS))'), colnames)
            println("plot times calculated for cell "*string(i))
            CSV.write("outputFiles_cell_cycle/"*mut_folder*"/plot_times/plot_times_"*string(i)*".csv", plot_times)
            CSV.write("outputFiles_cell_cycle/"*mut_folder*"/LastCellCyclecsv/sol_df"*string(i)*".csv", sol_SS)
    end
end

In [ ]:
function runSimulation(first_cell, last_cell)
    Threads.@threads for i in first_cell:last_cell
        # try a solve the cell and catch and continue if the cell fails
        try 
            solveCell(i)
        catch err
            println("cell "*string(i)*"failed, continuing.")
            #println(err)
            continue
        end
    end
end

In [ ]:
runSimulation(first_cell,last_cell)

In [ ]:
function getQuiescent(sol)
    sol_df = DataFrame(sol', syms)
    sol_NS = sol_df[!, :newSwitch]
    if(isequal(sol_NS[end], 1) == false)
        CSV.write("outputFiles_cell_cycle/"*mut_folder*"/quiescent/sol_df"*string(i)*".csv", sol_df)
        println("cell is in a quiescent state")
    else
        println("cell is not quiescent")
    end
end

In [ ]:
### using function to highlight quiescent cells

function solveCell(i)
    println("Starting solving cell: "*string(i)*".")
    #figure out the name of this cell's ode file
    odeName="odeModel_"*string(i)
    myFun=getfield(Main,Symbol(odeName))

    f=ODEFunction(myFun,syms=Symbol.(syms))
    y0=zeros(length(syms))
    y0=initConditionsCellCycle(y0,String.(syms))
    prob=ODEProblem(f,y0,(0.0,maxTimeSS))
    println("about to solve cell: "*string(i))
    
    sol = solve(prob,callback=cbs, abstol=1e-5,reltol=1e-3,saveat=1.0)
    
    if(sol.t[end]==maxTimeSS)   
        println("Cell: "*string(i)*" ended without finding a steady state in time.")
        getQuiescent(sol)
    else
        println("Cell: "*string(i)*" reached a steady state.")
        
        ### get steady state
        sol_df = DataFrame(sol', syms)
        time_stamp = sol.t
        sol_df[!, :time_stamp] = time_stamp
        indexOfLastCycleStart=findfirst(x->x>sol[indexOfMassTrackerTime2,end],sol.t)
        sol_SS = sol_df[indexOfLastCycleStart:end, :]
        
        # get SS time stamp
        time_stamp_SS = sol_SS[:, :time_stamp]
        
        ### Calculate times for cell cycle transitions
        plot_times = DataFrame(Array(collect(Get_CC_transitions(sol_SS, time_stamp_SS))'), colnames)
        println("plot times calculated for cell "*string(i))
        CSV.write("outputFiles_cell_cycle/"*mut_folder*"/plot_times/plot_times_"*string(i)*".csv", plot_times)
        CSV.write("outputFiles_cell_cycle/"*mut_folder*"/LastCellCyclecsv/sol_df"*string(i)*".csv", sol_SS)
    end
end